In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
df=pd.read_csv('../input/vehicle-dataset-from-cardekho/car data.csv')

In [ ]:
#Check Null
df.isnull().sum()

In [ ]:
df.describe()

In [ ]:
df.head()

In [ ]:
df.drop(['Car_Name'],axis=1,inplace=True)
df['Current Year']=2020
df['Used for'] = df['Current Year'] - df['Year']
df.drop(['Current Year'],axis=1,inplace=True)
df.drop(['Year'],axis=1,inplace=True)

In [ ]:
#one hot encoding
df=pd.get_dummies(df,drop_first=True)
df.head()

In [ ]:
x=df.iloc[:,1:]
y=df.iloc[:,0]
x.head()
y.head()

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)
from sklearn.ensemble import RandomForestRegressor
rf=RandomForestRegressor()
rf.fit(x_train, y_train)
predictions = rf.predict(x_test)

In [ ]:
import numpy as np
from sklearn import metrics
errors = abs(pred - y_test)
print('MAE:', round(metrics.mean_absolute_error(y_test, predictions),2))
print('MSE:', round(metrics.mean_squared_error(y_test, predictions),2))
print('RMSE:', round(np.sqrt(metrics.mean_squared_error(y_test, predictions)),2))
mape = 100 * (errors / y_test)
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

In [ ]:
#Herperparameter Tuning
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 30, num = 6)]
# max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 100]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 5, 10]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid,scoring='neg_mean_squared_error', n_iter = 10, cv = 10, verbose=2, random_state=42, n_jobs = 1)
rf_random.fit(x_train,y_train)

In [ ]:
rf_random.best_params_

In [ ]:
rf_opt=RandomForestRegressor(n_estimators= 1000,min_samples_split = 2,min_samples_leaf=1,max_features= 'sqrt',max_depth = 25)
rf_opt.fit(x_train, y_train)
pred = rf_opt.predict(x_test)

In [ ]:
# Calculate the absolute errors
error = abs(pred - y_test)
print('MAE:', metrics.mean_absolute_error(y_test, pred))
print('MSE:', metrics.mean_squared_error(y_test, pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, pred)))
mape = 100 * (error / y_test)
acuracy = 100 - np.mean(mape)
print('Accuracy:', round(acuracy, 2), '%.')